## ex 552


In [44]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_eager_execution()
from sklearn.datasets import make_moons
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
#housing
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
import tensorflow.compat.v1 as tf
import numpy as np

housing = fetch_california_housing()
samples,features = housing.data.shape
print(samples,features)
#housing_data_plus_bias = np.c_[np.ones((samples,1)),housing.data]

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_bias = np.c_[np.ones((samples,1)),scaled_housing_data]


tf.reset_default_graph()
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(samples / batch_size))

learning_rate = 0.01


X = tf.placeholder(tf.float32,shape=(None, features + 1),name='X')
y = tf.placeholder(tf.float32,shape=(None, 1),name='y')

theta = tf.Variable(tf.random_uniform([features + 1,1],-1,1,seed=42),name='theta')


# t = np.dot(theta.transpose(),X)
# p_hat = 1 / (1 + np.exp(-t)) # sigmoid(t)
# # Sigmoid Cross Entropy Cost Function 
# loss = tf.reduce_mean(-y * np.log(p_hat) - (1 - y) * np.log(1 - p_hat)).mean()

b = tf.Variable(tf.zeros([1])) 
print(X.shape,theta.shape)
t = tf.add(tf.matmul(X, theta), b) 
Y_hat = tf.nn.sigmoid(t) 
loss = tf.nn.sigmoid_cross_entropy_with_logits(logits = Y_hat, labels = y) 

gradients_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

traning_op = gradients_optimizer.minimize(loss)

init = tf.global_variables_initializer()


def fetch_batch(epoch,batch_index,batch_size):
    indices = np.random.randint(samples,size=batch_size)
    x_batch = scaled_housing_data_bias[indices]
    y_batch = housing.target.reshape(-1,1)[indices]
    return x_batch,y_batch


np.random.seed(0)

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        print ('epoch',epoch,'loss = ',
               loss.eval(feed_dict={X: scaled_housing_data_bias,
                                   y: housing.target.reshape(-1,1)}))
        for batch_index in range(n_batches):
            X_batch,y_batch = fetch_batch(epoch,batch_index,batch_size)
            sess.run(traning_op,feed_dict={X: X_batch,y: y_batch})
        
        
    best_theta = theta.eval()
    
print('best_theta')
print(best_theta)
            


In [50]:
# digits 
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
import tensorflow.compat.v1 as tf
import numpy as np


def logistic_regression():
    digits = load_digits()
    samples,features = digits.data.shape
    #print(samples,features)


    scaler = StandardScaler()
    scaled_digits = scaler.fit_transform(digits.data)
    scaled_digits_bias = np.c_[np.ones((samples,1)),scaled_digits]

    target = digits.target
    target[target != 5]  = 0
    target[target == 5]  = 1

    X_train, X_test, y_train, y_test = train_test_split(scaled_digits_bias, target, test_size=0.2, shuffle=False)
    #print('y_test',y_test)
    x_train_samples = X_train.shape[0]
    #print(x_train_samples)

    tf.reset_default_graph()
    n_epochs = 10
    batch_size = 100
    n_batches = int(np.ceil(x_train_samples / batch_size))

    learning_rate = 0.01


    X = tf.placeholder(tf.float32,shape=(None, features + 1),name='X')
    y = tf.placeholder(tf.float32,shape=(None, 1),name='y')

    theta = tf.Variable(tf.random_uniform([features + 1,1],-1,1,seed=15),name='theta')
    #b = tf.Variable(tf.ones([1])) 
    #print(X.shape,theta.shape)
    #t = tf.add(tf.matmul(X, theta), b) 
    t = tf.matmul(X, theta)
    Y_hat = tf.nn.sigmoid(t) 
    loss = tf.nn.sigmoid_cross_entropy_with_logits(logits = Y_hat, labels = y) 

    gradients_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

    traning_op = gradients_optimizer.minimize(loss)

    init = tf.global_variables_initializer()


    def fetch_batch(epoch,batch_index,batch_size):
        indices = np.random.randint(x_train_samples,size=batch_size)
        x_batch = X_train[indices]
        y_batch = y_train.reshape(-1,1)[indices]
        return x_batch,y_batch


    np.random.seed(0)

    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(n_epochs):
    #         print ('epoch',epoch,'loss = ',
    #                loss.eval(feed_dict={X: scaled_X_moons_bias,
    #                                    y: y_moons.reshape(-1,1)}))
            for batch_index in range(n_batches):
                X_batch,y_batch = fetch_batch(epoch,batch_index,batch_size)
                sess.run(traning_op,feed_dict={X: X_batch,y: y_batch})


        best_theta = theta.eval()
        y_pred_test = sess.run(Y_hat,feed_dict={X: X_test,y: y_test.reshape(-1,1)})

    #print('best_theta')
    #print(best_theta.shape)
    #print(y_pred_test)
    
    #evaluate


    y_pred_test[y_pred_test >= 0.5]  = 1
    y_pred_test[y_pred_test < 0.5]  = 0

    accur = accuracy_score(y_test,y_pred_test)
    print(accur)

logistic_regression()

0.9638888888888889
